In [ ]:
#!pip install facenet_pytorch

In [9]:
import os, cv2
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from facenet_pytorch import MTCNN
from tqdm import tqdm

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [10]:
mtcnn = MTCNN(keep_all=True, device=device)
new_img_dir = '/opt/ml/input/data/train/new_imgs'
img_path = '/opt/ml/input/data/train/images'

## define FaceCrop

In [21]:
import os, cv2
import torch
import pandas as pd
import numpy as np
from facenet_pytorch import MTCNN
from torchvision import transforms

def faceCrop(img, output_img_size : tuple = (224,224)): 
    '''
        image의 얼굴부분만 Crop 하는 함수
        img(ndarray) : cv2.imread와 cv2.cvtColor(,cv2.COLOR_BGR2RGB)의 결과값 입력
        output_img_size : output으로 받고싶은 image size 입력
    '''
    
    boxes,probs = mtcnn.detect(img) # MTCNN을 활용하여 img내에서 얼굴을 찾음
    # 찾은 경우 -> boxes : 좌표값, probs : 확률값 출력
    # 못찾은 경우 -> boxes : None, probs : [None] 출력
    
    if probs[0] == None: # 못찾은 경우, image center 값 출력
        print("Not Found")
        return img_center_crop(img, output_img_size)
    
    else :  # 찾은 경우, 얼굴 center기준으로 outputsize를 계산하여 출력
        xmid = int((boxes[0, 0]+boxes[0, 2])/2)
        ymid = int((boxes[0, 1]+boxes[0, 3])/2)
        
        xmin = xmid - output_img_size[0]//2
        ymin = ymid - output_img_size[1]//2
        xmax = xmid + output_img_size[0] - output_img_size[0]//2
        ymax = ymid + output_img_size[1] - output_img_size[1]//2
        
        if any((xmax < 0,xmax > img.shape[1],ymax < 0,ymax > img.shape[0])): # 찾은 후 box가 image 사이즈보다 클려
            return img_center_crop(img, output_img_size)
        
        else :  # 
            new_img = img[ymin:ymax, xmin:xmax, :]
            return new_img

def img_center_crop(img, output_img_size : tuple = (224,224)): 
    ximg_cen = int((img.shape[1])/2)
    yimg_cen = int((img.shape[0])/2)

    xmin = ximg_cen - output_img_size[0]//2
    ymin = yimg_cen - output_img_size[1]//2
    xmax = ximg_cen + output_img_size[0] - output_img_size[0]//2
    ymax = yimg_cen + output_img_size[1] - output_img_size[1]//2   
    
    new_img = img[ymin:ymax, xmin:xmax, :]
    
    return new_img

## FaceCrop 함수 Test

In [26]:
mtcnn = MTCNN(keep_all=True, device=device)
new_img_dir = '/opt/ml/input/data/train/new_imgs'
img_path = '/opt/ml/input/data/train/images'

cnt = 0
    
sub_dir = os.path.join(img_path, '000001_female_Asian_45')
    
for img_name in os.listdir(sub_dir):
    if img_name[0]=='.': continue
    
    img_dir = os.path.join(sub_dir, img_name)
    img = cv2.imread(img_dir)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    output_size = (224,224)
    img = faceCrop(img,output_size) #img input(x,y,3) -> output(224,224,3)
#     img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
#     plt.imshow(img)


    newpath = os.path.join(new_img_dir, sub_dir)
    cnt += 1
    print(os.path.join(newpath, img_name))
#     plt.imsave(os.path.join(path, img_name), img)
               
print(cnt)

/opt/ml/input/data/train/images/000001_female_Asian_45/mask4.jpg
/opt/ml/input/data/train/images/000001_female_Asian_45/mask1.jpg
/opt/ml/input/data/train/images/000001_female_Asian_45/mask2.jpg
/opt/ml/input/data/train/images/000001_female_Asian_45/mask3.jpg
/opt/ml/input/data/train/images/000001_female_Asian_45/normal.jpg
/opt/ml/input/data/train/images/000001_female_Asian_45/incorrect_mask.jpg
/opt/ml/input/data/train/images/000001_female_Asian_45/mask5.jpg
7


### unit test

In [ ]:
input_size = (224,224)

img_path = './input/data/train/images'
sub_dir = os.path.join(img_path, '006456_female_Asian_18')

for img_name in os.listdir(sub_dir):
    if img_name[0]=='.': continue
    
    img_dir = os.path.join(sub_dir, img_name)
    img = cv2.imread(img_dir)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
#     boxes,probs = mtcnn.detect(img)
#     print(img.shape)
#     print(boxes,probs)

    plt.imshow(img)

In [ ]:
input_size = (300,300)

img = cv2.imread("./input/data/train/images/006456_female_Asian_18/mask4.jpg")
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

# boxes,probs = mtcnn.detect(img)
# plt.imshow(img)
# print(img.shape)
# print(boxes,probs)
ximg_cen = int((img.shape[1])/2)
yimg_cen = int((img.shape[0])/2)

xmin = ximg_cen - input_size[0]//2
ymin = yimg_cen - input_size[1]//2
xmax = ximg_cen + input_size[0] - input_size[0]//2
ymax = yimg_cen + input_size[1] - input_size[1]//2   

# new_img = img[ymin:ymax, xmin:xmax, :]

# plt.imshow(new_img)

In [ ]:
any((1,2,1,0))

In [9]:
output_img_size = (224,224)

img = cv2.imread("./input/data/train/images/006456_female_Asian_18/mask4.jpg")
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
type(img)
# img = img_center_crop(img, input_size)

# tran = transforms.Compose([
#         transforms.CenterCrop((output_img_size[0]*1.5,output_img_size[1]*1.5)),
#         transforms.Resize((output_img_size[0],output_img_size[1])),
#         ])
# # img = torch.from_numpy(img)
# img=transforms.ToPILImage()(img)
# tran(img)

# tmp = os.path.join(new_img_dir, paths)
# cnt += 1
# plt.imsave(os.path.join(tmp, 'mask4.jpg'), img)
# # plt.imshow(new_img)

numpy.ndarray

In [ ]:
    tran = transforms.Compose([
        transforms.CenterCrop((output_img_size[0]*1.5,output_img_size[1]*1.5)),
        transforms.Resize((output_img_size[0],output_img_size[1]))
        ])